In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from reloading_utils import get_url

In [ ]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv(get_url('data/reloading_data - CATRIDGES.csv'))
print(cartridges)

cartridges_to_process = cartridges.loc[:, ['title', 'volume', 'diameter']].to_numpy()
print(cartridges_to_process)

# reloading_data - powder burning rate chart
powder_rates = pd.read_csv(get_url('data/reloading_data - powder burning rate chart.csv'))
print(powder_rates.head())

In [ ]:
def process_data_file(c, v, a):
    filename = f'data/reloading_data - %s.csv' % c
    try:
        d = pd.read_csv(get_url(filename))
        d['cartridge_caliber'] = c
        #d['cartridge_volume'] = v
        #d['cartridge_diameter'] = a
        return d
    except:
        print("ERROR: Cannot process:", filename)
        return None

frames = [ process_data_file(c, v, a) for c, v, a in cartridges_to_process ]
data = pd.concat(frames, ignore_index=True)

# print(data.head())  # DEBUG
print('data.shape', data.shape)

In [ ]:
data[~ np.isnan(data.press_psi)]

In [ ]:
lee = pd.DataFrame(columns=['bul_weight', 'bul_type', *data.columns.to_list()])
# print(101, lee.columns) #DEBUG

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = data.loc[pos].to_numpy()
    # print(123, line) # DEBUG
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (int(bul_weight), bul_type, *line)
        lee.loc[len(lee.index)] = line
print('lee.shape', lee.shape)

In [ ]:
lee

In [ ]:
lee = pd.merge(lee, cartridges, left_on='cartridge_caliber', right_on='title')

In [ ]:
lee.head()

In [ ]:
lee.shape

In [ ]:
#
# CHECK VALIDITY EXAMPLE
#
powders = pd.DataFrame({"key": ["#1", "#2", "#3", "#4", '#5'], "value": [10, 20, 30, 40, 50]})
print(powders)

In [ ]:
data = pd.DataFrame({"powder": ["#1", "#2", "#3", "#4", '#xz'], "load_min": [10, 20, 30, 40, 22 ], 'load_max': [20, 25, 35, 45, 42]})
print(data)

In [ ]:
# INNER JOIN
# SELECT *
# FROM df1
# INNER JOIN df2
#   ON df1.key = df2.key;

pd.merge(powders, data, left_on='key', right_on='powder')

In [ ]:
# LEFT OUTER JOIN
# Show all records from df1.

# SELECT *
# FROM df1
# LEFT OUTER JOIN df2
#  ON df1.key = df2.key;
# pd.merge(df1, df2, on="key", how="left")

left_outer_join = pd.merge(powders, data, left_on='key', right_on='powder', how="left")[['key', 'powder']]
print(left_outer_join)

print(left_outer_join[['key', 'powder']])

# WHERE
# Filtering in SQL is done via a WHERE clause.

# SELECT *
# FROM tips
# WHERE time = 'Dinner';

# tips[tips["total_bill"] > 10]

print(left_outer_join[pd.isna(left_outer_join.powder)])

print('# for loop...')
for i in left_outer_join[pd.isna(left_outer_join.powder)][['key']].to_numpy():
    powder_key = i[0]
    print("Never used in data:", powder_key)


In [ ]:
# RIGHT JOIN
# Show all records from df2.

# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

right_join = pd.merge(powders, data, left_on='key', right_on='powder', how="right")
print(right_join)

In [ ]:
# WHERE
# Filtering in SQL is done via a WHERE clause.

# SELECT *
# FROM tips
# WHERE time = 'Dinner';

# tips[tips["total_bill"] > 10]



In [ ]:
# Show all powders never used in data tables

print(lee.head())
print(powder_rates.columns.to_list())
print(lee.columns.to_list())

In [ ]:
left_outer_join = pd.merge(powder_rates, lee, on='powder', how="left")[['powder', 'bul_weight']]
print(left_outer_join[left_outer_join.powder == 'CLAYS'])

In [ ]:
for i in left_outer_join[pd.isna(left_outer_join.bul_weight)].to_numpy():
    powder_key = i[0]
    print("Never used in data:", powder_key)

In [ ]:
print(left_outer_join[pd.isna(left_outer_join.bul_weight)])

In [ ]:
# RIGHT JOIN
# Show all records from df2.

# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

right_join = pd.merge(powder_rates, lee, on='powder', how="right")[['powder', 'rate']]
print(right_join)

right_join[pd.isna(right_join.rate)]

In [1]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

from reloading_utils import get_url

In [2]:
lee = pd.read_csv('lee.csv', index_col=0)

In [3]:
lee.head()

,bul_weight,bul_type,powder,s_load,s_velocity,ne_load,ne_velocity,press,units,oal,cartridge_caliber,cartridge_volume,cartridge_diameter,powder_id,bul_type_id,powder_rate
0,115,jacketed,v-N350,5.4,1129.0,6.5,1293.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,58,0,20
1,115,jacketed,v-3N37,5.6,1129.0,6.5,1289.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,54,0,19
2,115,jacketed,v-N330,4.5,1076.0,5.4,1227.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,40,0,15
3,115,jacketed,v-N340,4.8,1129.0,5.4,1220.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,46,0,18
4,115,jacketed,wSUPER-FLD,4.9,1060.0,5.7,1195.0,31900.0,PSI,1.169,9mm parabellum,0.74,0.356,55,0,19


In [4]:
lee[(lee.bul_weight == 115) & (lee.powder == 'v-N340') ][['s_load']]

,s_load
3,4.8
41,5.0
54,5.2


In [5]:
def s_load(bul_weight, bul_type, powder):
    d = lee[(lee.bul_weight == bul_weight) & (lee.bul_type == bul_type) & (lee.powder == powder)][['s_load']]
    arr = d.to_numpy()
    return arr[0][0]


In [6]:
print(s_load(115, 'jacketed', 'v-N340'))

4.8


In [28]:
grp = lee[lee.bul_type == 'XTP'].groupby(['bul_weight', 'bul_type'])

In [29]:
for g in grp:
    print(g[0], len(g[1]))

(110, 'XTP') 26
(115, 'XTP') 24
(125, 'XTP') 50


In [30]:
powder_fractions = pd.DataFrame(columns = ['bul_weight', 'bul_type', 'powder1', 's_load1', 'powder2', 's_load2', 'fraction'])
for g in list(grp)[:5]:
    bul_weight, bul_type = g[0]
    # print(bul_weight, bul_type)
    gg = g[1]
    powders = gg['powder'].to_list()
    powder_pairs = itertools.combinations(powders, 2)
    # print(list(powder_pairs))
    for p1, p2 in powder_pairs:
        l1 = s_load(bul_weight, bul_type, p1)
        assert(type(l1) is np.float64)
        l2 = s_load(bul_weight, bul_type, p2)
        assert(type(l2) is np.float64)
        powder_fractions.loc[len(powder_fractions.index)] = [bul_weight, bul_type, p1, l1, p2, l2, l1 / l2]
    

In [31]:
powder_fractions.shape

(1826, 7)

In [32]:
powder_fractions

,bul_weight,bul_type,powder1,s_load1,powder2,s_load2,fraction
0,110,XTP,v-3N37,7.3,v-N350,6.6,1.106061
1,110,XTP,v-3N37,7.3,CFE PISTOL,6.5,1.123077
2,110,XTP,v-3N37,7.3,v-N320,5.4,1.351852
3,110,XTP,v-3N37,7.3,v-N340,6.2,1.177419
4,110,XTP,v-3N37,7.3,R SILHOUETTE,6.3,1.158730
...,...,...,...,...,...,...,...
1821,125,XTP,UNIQUE,5.0,IMR PB,4.5,1.111111
1822,125,XTP,UNIQUE,5.0,ACCUR #2,4.3,1.162791
1823,125,XTP,v-N310,6.0,IMR PB,4.5,1.333333
1824,125,XTP,v-N310,6.0,ACCUR #2,4.3,1.395349


In [33]:
ppf_table = powder_fractions[['powder1', 'powder2', 'fraction']].copy()

In [34]:
ppf_table

,powder1,powder2,fraction
0,v-3N37,v-N350,1.106061
1,v-3N37,CFE PISTOL,1.123077
2,v-3N37,v-N320,1.351852
3,v-3N37,v-N340,1.177419
4,v-3N37,R SILHOUETTE,1.158730
...,...,...,...
1821,UNIQUE,IMR PB,1.111111
1822,UNIQUE,ACCUR #2,1.162791
1823,v-N310,IMR PB,1.333333
1824,v-N310,ACCUR #2,1.395349


In [ ]:
import itertools
# import pprint

l1 = ['a', 'b', 'c']
l2 = ['X', 'Y', 'Z']

p = itertools.product(l1, l2)

print(p)
# <itertools.product object at 0x1026edd80>

print(type(p))
# <class 'itertools.product'>

In [ ]:
for v in p:
    print(v)

In [ ]:
list(p)

In [ ]:
p = itertools.combinations(l1, 2)

list(p)